# Feature selection

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/yggdrasil-decision-forests/blob/main/documentation/public/docs/tutorial/feature_selection.ipynb)

## Setup

In [ ]:
pip install ydf -U

## What is feature selection?

**Feature selection** is the process of automatically identifying and removing unnecessary input features to improve the quality, inference speed, and subsequent training speed of a model. In YDF, feature selection is enabled by setting the `feature_selector` learner argument.

Training a model with feature selection is slower than training a model without feature selection as the former involves training and processing multiple intermediate models. Therefore, feature selection is generally only done once at the end of the model development after model hyper-parameter and input features have been determined.

In YDF, the feature selection algorithm is "guided" by two values: A quality metric to optimize (e.g., accuracy, logloss, AUC) and a variable importance metric (e.g., mean decrease in accuracy). Those are selected automatically unless manually specified by the user.


## This tutorial

In this tutorial, we train and compare two models: One without and one with feature selection on the [christine dataset](https://www.openml.org/search?type=data&sort=runs&id=41142&status=active).

## Prepare dataset

In [1]:
import io
import urllib.request
import pandas as pd
from scipy.io import arff
from sklearn.model_selection import train_test_split
import ydf

In [2]:
with urllib.request.urlopen(
    "https://www.openml.org/data/download/19335515/file764d5d063390.arff"
) as response:
  raw_data = arff.loadarff(io.StringIO(response.read().decode("utf-8")))
dataset = pd.DataFrame(raw_data[0])
dataset.head(5)  # Print the first 5 examples

,class,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V1627,V1628,V1629,V1630,V1631,V1632,V1633,V1634,V1635,V1636
0,b'0',443.0,375.0,109.0,423.0,534.0,296.0,82.0,571.0,132.0,...,174.0,23.0,13.0,167.0,168.0,286.0,517.0,388.0,197.0,331.0
1,b'1',486.0,716.0,136.0,231.0,705.0,343.0,241.0,501.0,174.0,...,147.0,241.0,62.0,278.0,208.0,167.0,282.0,222.0,0.0,380.0
2,b'1',277.0,424.0,75.0,264.0,639.0,172.0,87.0,600.0,100.0,...,355.0,33.0,21.0,237.0,111.0,109.0,433.0,329.0,0.0,299.0
3,b'0',220.0,459.0,117.0,280.0,767.0,286.0,187.0,684.0,141.0,...,303.0,376.0,45.0,222.0,107.0,360.0,271.0,381.0,0.0,329.0
4,b'1',288.0,414.0,121.0,285.0,556.0,356.0,105.0,739.0,205.0,...,361.0,0.0,8.0,133.0,150.0,176.0,656.0,408.0,293.0,521.0


In [3]:
print(
    f"The dataset contains {dataset.shape[1]} features and"
    f" {dataset.shape[0]} rows."
)

The dataset contains 1637 features and 5418 rows.


Split the dataset into a training and testing dataset.

In [4]:
train_dataset, test_dataset = train_test_split(
    dataset, test_size=0.3, random_state=1234
)

## Train a model without feature selection

Let's train and evaluate a model with all the default parameter values.

In [5]:
model_1 = ydf.RandomForestLearner(label="class").train(train_dataset)

evaluation_1 = model_1.evaluate(test_dataset)

print("Accuracy:", evaluation_1.accuracy)
print("Number of features:", len(model_1.input_features()))

[Warning] Column 'V496' is detected as CATEGORICAL but its values look like numbers (e.g., 0, 1). Should the column not be NUMERICAL instead? If so, feed numerical values instead of strings or objects.
Column 'V566' is detected as CATEGORICAL but its values look like numbers (e.g., 0, 1). Should the column not be NUMERICAL instead? If so, feed numerical values instead of strings or objects.
Column 'V588' is detected as CATEGORICAL but its values look like numbers (e.g., 0, 1). Should the column not be NUMERICAL instead? If so, feed numerical values instead of strings or objects.
Column 'V685' is detected as CATEGORICAL but its values look like numbers (e.g., 0, 1). Should the column not be NUMERICAL instead? If so, feed numerical values instead of strings or objects.
Column 'V741' is detected as CATEGORICAL but its values look like numbers (e.g., 0, 1). Should the column not be NUMERICAL instead? If so, feed numerical values instead of strings or objects.
Column 'V951' is detected as C

Train model on 3792 examples
Model trained in 0:00:02.416616
Accuracy: 0.7170971709717097
Number of features: 1636


With all 1636 input features, the model's accuracy is 0.7171

## Train a model with feature selection

Now, let's train a similar model with feature selection.

In [6]:
model_2 = ydf.RandomForestLearner(
    label="class",
    # Tell the Random Forest to compute Out-of-bag variable importances.
    # This is expensive but gives the best results.
    compute_oob_variable_importances=True,
    # Enable feature selection using the "backward selection" algorithm.
    feature_selector=ydf.BackwardSelectionFeatureSelector(),
).train(train_dataset)

evaluation_2 = model_2.evaluate(test_dataset)

print("Accuracy:", evaluation_2.accuracy)
print("Number of features:", len(model_2.input_features()))

Run backward feature selection on 1636 features
[Iteration 0] Train model on 1636 features
Optimizing metric "loss". The available metrics are ['loss', 'num_examples_weighted', 'accuracy']
[Iteration 0] Score:-0.551959 Metrics:{'loss': 0.5519588122415486, 'num_examples_weighted': 3792.0, 'accuracy': 0.7241561181434599}
Guide feature selection using "MEAN_DECREASE_IN_AUC_1_VS_OTHERS" variable importance. The available variable importances are ['MEAN_DECREASE_IN_PRAUC_1_VS_OTHERS', 'MEAN_DECREASE_IN_ACCURACY', 'MEAN_DECREASE_IN_AP_1_VS_OTHERS', 'NUM_AS_ROOT', 'SUM_SCORE', 'MEAN_DECREASE_IN_AUC_1_VS_OTHERS', 'NUM_NODES', 'INV_MEAN_MIN_DEPTH']
[Iteration 1] Train model on 1473 features
Train model on 3792 examples
Model trained in 0:01:54.265457
[Iteration 1] Score:-0.546692 Metrics:{'loss': 0.5466921646462337, 'num_examples_weighted': 3792.0, 'accuracy': 0.7310126582278481}
[Iteration 2] Train model on 1326 features
Train model on 3792 examples
Model trained in 0:01:42.468182
[Iteration 2

This model take much longer to train. However, this model only uses 81 features (the previous model used 1636 features) and has a better accuracy of 0.7478 (the previous model had an accuracy of 0.7170).

The feature selection logs are available in the "Feature selection" tab of the model description:

**Click on the "Feature selection" tab**

In [7]:
model_2.describe()

## Feature selection of a model with cheaper variable importances

The previous model took some time to train. The main reason is that out-of-bag random forest variable importance (enabled with `compute_oob_variable_importances=True`) is expensive to compute. Alternatively, it is possible to do feature selection with cheap structural variable importances (though, the results are not as good):

In [8]:
model_3 = ydf.RandomForestLearner(
    label="class",
    # Enable feature selection
    feature_selector=ydf.BackwardSelectionFeatureSelector(
        # Allow to fallback on structural variable importance
        # if not other variable importance is available.
        allow_structural_variable_importance=True
    ),
).train(train_dataset)

evaluation_3 = model_3.evaluate(test_dataset)

print("Accuracy:", evaluation_3.accuracy)
print("Number of features:", len(model_3.input_features()))

Run backward feature selection on 1636 features
[Iteration 0] Train model on 1636 features
Optimizing metric "loss". The available metrics are ['loss', 'num_examples_weighted', 'accuracy']
[Iteration 0] Score:-0.551959 Metrics:{'loss': 0.5519588122415486, 'num_examples_weighted': 3792.0, 'accuracy': 0.7241561181434599}
Guide feature selection using "INV_MEAN_MIN_DEPTH" variable importance. The available variable importances are ['NUM_NODES', 'SUM_SCORE', 'NUM_AS_ROOT', 'INV_MEAN_MIN_DEPTH']
[Iteration 1] Train model on 1439 features
Train model on 3792 examples
Model trained in 0:00:02.142809
[Iteration 1] Score:-0.550163 Metrics:{'loss': 0.5501630669081473, 'num_examples_weighted': 3792.0, 'accuracy': 0.7209915611814346}
[Iteration 2] Train model on 1296 features
Train model on 3792 examples
Model trained in 0:00:01.947157
[Iteration 2] Score:-0.551167 Metrics:{'loss': 0.5511673348405661, 'num_examples_weighted': 3792.0, 'accuracy': 0.7212552742616034}
[Iteration 3] Train model on 116

We see that this model as an accuracy of 0.726322 which is better than the model without feature selection (0.7170) but worst than the model with the OOB variable importances (0.7478).
Also, less features were removed (the final model has 198 features instead of 81).

## To go further

In those examples, we showed how to use the Random Forest learner. Feature selection is also possible with other learners such as Gradient Boosted Trees and CART. Notably, while Random Forest is generally better than Gradient Boosted Trees on datasets with a small number of examples compared to the number of input features, feature selection on Gradient Boosted Trees is faster.

In the example above, the models were trained without a validation dataset. Instead, if you provide a validation dataset (using the `valid` argument of the `train` method), the objective metric and variable importance will be computed on this validation dataset. This is especially useful if you suspect your dataset has a distribution shift. Using a validation dataset is also significantly faster for the Random Forest learner. 
